In [6]:
import pandas as pd
import json
import re
import os
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /afs/crc.nd.edu/user/k/kmealey2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
data = json.load(open('../../OMIn_dataset/data/FAA_data/faa.json'))

In [4]:
len(data)

2748

In [14]:
formatted = []
doc_keys = list(data.keys())
for i in range(len(data)):
    tokenized = []
    sentences = nltk.sent_tokenize(data[doc_keys[i]][0])
    for isentence, sentence in enumerate(sentences):
        words = nltk.word_tokenize(sentence)
        tokenized.append(words)
    placehold = [[]]*len(sentences)
    formatted.append({'sentences':tokenized, 'ner':placehold, 'relations':placehold, 'doc_key':doc_keys[i]})

In [16]:
formatted[1]

{'sentences': [['tow', 'plane', 'became', 'airborne', 'then', 'settled', '.'],
  ['student', 'thought', 'tow', 'in', 'trouble', '&', 'released', '.'],
  ['hit', 'tree', '.']],
 'ner': [[], [], []],
 'relations': [[], [], []],
 'doc_key': 'faa/1_19750419011349A'}

In [17]:
with open("./faa_plmarker.json", "w") as json_file:
    for i in range(len(formatted)):
        formatted_str = json.dumps(formatted[i]) + '\n'
        json_file.write(formatted_str)